### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [30]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [31]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [32]:
keywords = ["connectThrashCheck", "foldersInProfileTab", "inviteFriends", "tabletSupport", 
            "moveOrCopy", "newTeamBuildingForChatAllowMakeTeam", "teamInvites", "teamsRedesign", 
            "webOfTrust", "whyDidYouRender", "FeatureFlags"]
# removing "admin" because it affects about 100 files 
# and it is tricky to detect if it is really the feature flag that is involved

# adding FeatureFlags because some featureflags might be accessed through the table of feature flags

### List files with keywords

#### List all .go files

In [33]:
go_folders = [x[0] for x in os.walk("./client/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./client/go/vendor/gopkg.in/src-d/go-git.v4/plumbing/format/pktline/scanner.go
./client/go/vendor/github.com/couchbase/vellum/vellum_mmap.go
./client/go/client/cmd_chat_createchannel.go
./client/go/kbfs/simplefs/upload_manager.go
./client/packaging/linux/tuxbot/bot/chatbot/logger.go
./client/go/kbfs/libkbfs/config_mock_test.go
./client/go/libkb/skb_test.go
./client/go/libkb/resolve.go
./client/go/vendor/github.com/syndtr/goleveldb/leveldb/db_write.go
./client/go/client/cmd_account_reset.go


In [34]:
len(go_files)

4744

#### List the interesting files, containing at least one keyword with the featureflag 

In [35]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./client/go/stellar/stellarsvc/service.go contains :
-> FeatureFlags


./client/go/keybase/main.go contains :
-> FeatureFlags


./client/go/engine/features_test.go contains :
-> FeatureFlags


./client/go/libcmdline/cmdline.go contains :
-> FeatureFlags


./client/go/protocol/keybase1/notify_users.go contains :
-> webOfTrust


./client/go/protocol/keybase1/invite_friends.go contains :
-> inviteFriends


./client/go/externals/services.go contains :
-> FeatureFlags


./client/go/teams/chain.go contains :
-> teamInvites


./client/go/teams/hidden/hidden.go contains :
-> FeatureFlags


./client/go/chat/journey_card_manager.go contains :
-> FeatureFlags


./client/go/libkb/chain_link_v2.go contains :
-> FeatureFlags


./client/go/libkb/test_common.go contains :
-> FeatureFlags


./client/go/libkb/features.go contains :
-> FeatureFlags


./client/go/libkb/globals.go contains :
-> FeatureFlags


./client/go/libkb/config.go contains :
-> FeatureFlags


./client/go/libkb/env.go contains :
-> Fe

In [36]:
len(go_file_interests)

21

## Step 3 - Search Feature Toggles dependencies in the AST

In [37]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./client/go/stellar/stellarsvc/service.go 



File :             ./client/go/keybase/main.go 



if !e.GetFeatureFlags().Admin(e.GetUID()) {
		// Admin only for now
		return
	}


-> !e.GetFeatureFlags().Admin(e.GetUID())


File :             ./client/go/engine/features_test.go 



File :             ./client/go/libcmdline/cmdline.go 



File :             ./client/go/protocol/keybase1/notify_users.go 



File :             ./client/go/protocol/keybase1/invite_friends.go 



File :             ./client/go/externals/services.go 



File :             ./client/go/teams/chain.go 



if prevState.IsImplicit() {
			// Check to see if the additions were previously members of the team
			checkImpteamInvites := func() error {
				addedUIDs := make(map[keybase1.UID]bool)
				for _, invites := range additions {
					for _, invite := range invites {
						cat, err := invite.Type.C()
						if err != nil {
							return err
						}
						if cat == keybase1.TeamInviteCategory_KEYBA

In [38]:
type_nodes

{'comment': 127,
 'package_clause': 1,
 'package': 1,
 'package_identifier': 565,
 '\n': 780,
 'import_declaration': 1,
 'import': 1,
 'import_spec_list': 1,
 '(': 795,
 'import_spec': 13,
 'interpreted_string_literal': 13,
 '"': 26,
 ')': 795,
 'type_declaration': 111,
 'type': 111,
 'type_spec': 111,
 'type_identifier': 1787,
 'interface_type': 108,
 'interface': 108,
 'method_spec_list': 108,
 '{': 127,
 'method_spec': 562,
 'field_identifier': 623,
 'parameter_list': 791,
 'parameter_declaration': 1297,
 'qualified_type': 558,
 '.': 558,
 ',': 725,
 'slice_type': 69,
 '[': 79,
 ']': 79,
 '}': 127,
 'pointer_type': 89,
 '*': 89,
 'map_type': 9,
 'map': 9,
 'struct_type': 19,
 'struct': 19,
 'field_declaration_list': 19,
 'identifier': 589,
 'variadic_parameter_declaration': 16,
 '...': 16,
 'field_declaration': 61,
 'raw_string_literal': 4,
 'function_type': 7,
 'func': 7,
 'const_declaration': 3,
 'const': 3,
 'const_spec': 8,
 '=': 4,
 'expression_list': 4,
 'array_type': 1,
 'int

In [14]:
# Pattern matching -> does not work yet

query = GO_LANGUAGE.query("""
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
""")

captures = query.captures(tree.root_node)

